# Demonstration of [dplyr issue 1363](https://github.com/hadley/dplyr/issues/1363)

In [1]:
library(dplyr, warn=FALSE)
library(readr)
library(tidyr)

sessionInfo()

R version 3.2.1 (2015-06-18)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: OS X 10.10.5 (Yosemite)

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] tidyr_0.2.0      readr_0.1.0.9000 dplyr_0.4.2.9002

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.0     digest_0.6.8    assertthat_0.1  IRdisplay_0.3  
 [5] repr_0.2        R6_2.1.1        jsonlite_0.9.16 DBI_0.3.1      
 [9] magrittr_1.5    evaluate_0.7    stringi_0.5-2   uuid_0.1-1     
[13] IRkernel_0.4    tools_3.2.1     stringr_1.0.0   parallel_3.2.1 
[17] base64enc_0.1-2 rzmq_0.7.7     

In [2]:
platforms = c(
  'snps_affy500_log'='Affymetrix 500K Set',
  'snps_hh550_log'='Illumina HumanHap550',
  'snps_ho1_log'='Illumina HumanOmni1',
  'snps_exac_log'='ExAC',
  'snps_kg3_log'='1000 Genomes Phase 3'
)
platforms

snps_affy500_log         snps_hh550_log           snps_ho1_log 
 "Affymetrix 500K Set" "Illumina HumanHap550"  "Illumina HumanOmni1" 
         snps_exac_log           snps_kg3_log 
                "ExAC" "1000 Genomes Phase 3"

In [3]:
combined_df = readr::read_tsv('data/combined.tsv.gz')
head(combined_df, 2)

,metaedge,abbreviation,entrez_gene_id,symbol,node,degree,snps_hh550,snps_ho1,snps_affy500,snps_exac,snps_kg3,degree_log,snps_hh550_log,snps_ho1_log,snps_affy500_log,snps_exac_log,snps_kg3_log
1,gene - binding - compound,GbC,1,A1BG,gene::1,0,4,12,1,6,38,0,0.69897,1.1139,0.30103,0.8451,1.5911
2,gene - upregulation - compound,GuC,1,A1BG,gene::1,0,4,12,1,6,38,0,0.69897,1.1139,0.30103,0.8451,1.5911


In [4]:
gathered_df = combined_df %>%
  dplyr::select(-(snps_hh550:snps_kg3)) %>%
  tidyr::gather(platform, snps_log, snps_hh550_log:snps_kg3_log, convert = FALSE) %>%
  # dplyr::mutate(platform = as.character(platform)) %>% # converting platform to character resolves the issue
  dplyr::mutate(platform_name = platforms[platform])

head(gathered_df, 2)

,metaedge,abbreviation,entrez_gene_id,symbol,node,degree,degree_log,platform,snps_log,platform_name
1,gene - binding - compound,GbC,1,A1BG,gene::1,0,0,snps_hh550_log,0.69897,Affymetrix 500K Set
2,gene - upregulation - compound,GuC,1,A1BG,gene::1,0,0,snps_hh550_log,0.69897,Affymetrix 500K Set


In [5]:
class(gathered_df$platform)

[1] "factor"

In [6]:
gathered_df %>%
  dplyr::select(platform, platform_name) %>%
  dplyr::distinct()

,platform,platform_name
1,snps_hh550_log,Affymetrix 500K Set
2,snps_ho1_log,Illumina HumanHap550
3,snps_affy500_log,Illumina HumanOmni1
4,snps_exac_log,ExAC
5,snps_kg3_log,1000 Genomes Phase 3


In [7]:
gathered_df$platform[1]

[1] snps_hh550_log
5 Levels: snps_hh550_log snps_ho1_log snps_affy500_log ... snps_kg3_log

In [8]:
as.integer(gathered_df$platform[1])

[1] 1

In [9]:
platforms[gathered_df$platform[1]]

snps_affy500_log 
"Affymetrix 500K Set"